In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

In [2]:
spark_conf = SparkConf()
spark_conf.setMaster("spark://master:7077")
spark_conf.setAppName("GCSExample")
spark_conf.set("spark.driver.memory", "2g")
spark_conf.set("spark.executor.cores", "1")
spark_conf.set("spark.driver.cores", "1")

In [3]:
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

In [4]:
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

In [5]:
#  Google Storage File Path
gsc_file_path = "gs://delabs-lab6-input/sales.csv"

# Create data frame
df = spark.read.format("csv").option("header", "true").load(gsc_file_path)
df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- time_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- promotion_id: string (nullable = true)
 |-- store_id: string (nullable = true)
 |-- store_sales: string (nullable = true)
 |-- store_cost: string (nullable = true)
 |-- unit_sales: string (nullable = true)



In [6]:
new_df = df.select("unit_sales")

new_df.show()
df.show()

+----------+
|unit_sales|
+----------+
|         2|
|         3|
|         1|
|         3|
|         2|
|         3|
|         2|
|         2|
|         2|
|         1|
|         2|
|         2|
|         2|
|         1|
|         1|
|         2|
|         2|
|         1|
|         3|
|         1|
+----------+
only showing top 20 rows

+----------+-------+-----------+------------+--------+-----------+----------+----------+
|product_id|time_id|customer_id|promotion_id|store_id|store_sales|store_cost|unit_sales|
+----------+-------+-----------+------------+--------+-----------+----------+----------+
|       337|    371|       6280|        null|       2|     1.5000|    0.5100|         2|
|      1512|    371|       6280|        null|       2|     1.6200|    0.6318|         3|
|       963|    371|       4018|        null|       2|     2.4000|    0.7200|         1|
|       181|    371|       4018|        null|       2|     2.7900|    1.0323|         3|
|      1383|    371|       4018|       

how many distinct products have been sold in each store? 


In [7]:
dfw = df.select("product_id","store_id").distinct()
dfw.show(2)

+----------+--------+
|product_id|store_id|
+----------+--------+
|       955|       2|
|      1119|       2|
+----------+--------+
only showing top 2 rows



In [8]:
dfw.write.format("csv").save("gs://delabs-lab6-output/lab61.csv")

In [9]:
# Stop the spark context
spark.stop()